In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

In [3]:

# parquet files generated from indexing pipeline
INPUT_DIR = "C:/Users/rishu/OneDrive/Desktop/Rishub/AiHCCC/agents-grant/grant_agent_2/graphrag/ragtest/output/20240704-184122/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [4]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 32


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,10,# SSHRC Doctoral Fellowships and Research Prop...,1,7.5,SSHRC Doctoral Fellowships and Research Proposals,The impact severity rating is high due to the ...,The community revolves around SSHRC Doctoral F...,[{'explanation': 'SSHRC Doctoral Fellowships p...,"{\n ""title"": ""SSHRC Doctoral Fellowships an...",a97f9429-b24b-4d3f-81b7-bce2bfa5a865
1,11,# Ontario Graduate Scholarship and Academic Tr...,1,7.5,Ontario Graduate Scholarship and Academic Tran...,The impact severity rating is high due to the ...,The community is centered around the Ontario G...,[{'explanation': 'The Ontario Graduate Scholar...,"{\n ""title"": ""Ontario Graduate Scholarship ...",0684409b-fac7-4ac6-803a-407a8f6ef84b
2,12,# Canadian Liver Foundation and Health Grants\...,1,7.5,Canadian Liver Foundation and Health Grants,The impact severity rating is high due to the ...,The community is centered around the Canadian ...,[{'explanation': 'The Canadian Liver Foundatio...,"{\n ""title"": ""Canadian Liver Foundation and...",414e537f-1b23-4f2a-b131-6d9ec3884c4b
3,13,# Indigenous Languages and Cultures Program Co...,1,7.5,Indigenous Languages and Cultures Program Comm...,The impact severity rating is high due to the ...,The Indigenous Languages and Cultures Program ...,[{'explanation': 'The Indigenous Languages and...,"{\n ""title"": ""Indigenous Languages and Cult...",fb3c0ebc-c983-4eb9-b1b4-d8b41ae7826c
4,14,# Canadian Heritage and Cultural Funding\n\nTh...,1,8.5,Canadian Heritage and Cultural Funding,The impact severity rating is high due to the ...,The community revolves around Canadian Heritag...,[{'explanation': 'Canadian Heritage offers a w...,"{\n ""title"": ""Canadian Heritage and Cultura...",a15a6886-81c0-49e0-8c1e-25325eb720f3


## Setting Up Global Search

In [5]:
context_builder_global = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

context_builder_params_global = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params_global = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params_global = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

search_engine_global = GlobalSearch(
    llm=llm,
    context_builder=context_builder_global,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params_global,
    reduce_llm_params=reduce_llm_params_global,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params_global,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [6]:
result = await search_engine_global.asearch(
    "Can you tell me the best grants for women in healthcare sector?"
)

print(result.response)
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")
result.context_data["reports"]

### Grants Supporting Women in the Healthcare Sector

In the realm of grants supporting women in the healthcare sector, several opportunities stand out based on the analysis of multiple reports from different analysts.

#### Women Entrepreneurship Fund by Global Affairs Canada [Data: Reports (20)]
The Women Entrepreneurship Fund, sponsored by Global Affairs Canada, is a notable initiative specifically tailored to empower and promote women in entrepreneurship within the healthcare sector. This fund provides crucial financial support to women entrepreneurs, aiming to foster their growth and success in healthcare-related ventures.

#### Canadian Heritage and Cultural Funding Programs [Data: Reports (14)]
The Canadian Heritage and Cultural Funding programs offer a diverse range of funding opportunities that extend to healthcare initiatives. While not exclusively focused on women, these programs reflect a commitment to supporting projects related to Canadian culture and heritage, which may 

,id,title,occurrence weight,content,rank
0,16,Government of Canada and Financial Support Pro...,1.000000,# Government of Canada and Financial Support P...,8.5
1,7,Canadian Innovation Community,0.846154,# Canadian Innovation Community\n\nThe Canadia...,8.5
2,14,Canadian Heritage and Cultural Funding,0.538462,# Canadian Heritage and Cultural Funding\n\nTh...,8.5
3,19,Government of Canada and Business Entities,0.538462,# Government of Canada and Business Entities\n...,8.5
4,21,Canada's Grant Programs Community,0.538462,# Canada's Grant Programs Community\n\nThe com...,7.5
5,20,"Women Entrepreneurship Fund and Innovation, Sc...",0.384615,"# Women Entrepreneurship Fund and Innovation, ...",7.5
6,31,CDHA and Sponsorship Community,0.307692,# CDHA and Sponsorship Community\n\nThe commun...,7.5
7,28,Grant Application Entities and Requirements,0.307692,# Grant Application Entities and Requirements\...,7.5
8,25,Canadian Businesses and COVID-19 Support Commu...,0.307692,# Canadian Businesses and COVID-19 Support Com...,7.5
9,17,Canada Small Business Financing Program and Go...,0.307692,# Canada Small Business Financing Program and ...,7.5


In [7]:
result = await search_engine_global.asearch(
    "Can you write a grant proposal for a small business in the health care sector? like the CIHR foundation grant."
)

print(result.response)
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")
result.context_data["reports"]

### Grant Proposal for a Small Business in the Health Care Sector

In preparing a grant proposal for a small business in the health care sector, it is essential to consider various funding opportunities and align the proposal with the specific requirements of the granting organization. Drawing insights from multiple analysts who have highlighted key aspects related to grants and funding in the health care sector, we can craft a comprehensive grant proposal tailored to meet the criteria of organizations like the CIHR foundation grant.

#### Proposal Summary:
Our small business, focused on innovative health care solutions, seeks funding to support research and development initiatives aimed at advancing cardiovascular health. With a strong emphasis on collaboration and innovation, we aim to leverage partnerships with academic institutions and non-profits to drive research advancements in cardiovascular care.

#### Key Components of the Proposal:
1. **Collaboration Agreement (Data: Reports

,id,title,occurrence weight,content,rank
0,16,Government of Canada and Financial Support Pro...,1.000000,# Government of Canada and Financial Support P...,8.5
1,7,Canadian Innovation Community,0.846154,# Canadian Innovation Community\n\nThe Canadia...,8.5
2,14,Canadian Heritage and Cultural Funding,0.538462,# Canadian Heritage and Cultural Funding\n\nTh...,8.5
3,19,Government of Canada and Business Entities,0.538462,# Government of Canada and Business Entities\n...,8.5
4,21,Canada's Grant Programs Community,0.538462,# Canada's Grant Programs Community\n\nThe com...,7.5
5,20,"Women Entrepreneurship Fund and Innovation, Sc...",0.384615,"# Women Entrepreneurship Fund and Innovation, ...",7.5
6,31,CDHA and Sponsorship Community,0.307692,# CDHA and Sponsorship Community\n\nThe commun...,7.5
7,28,Grant Application Entities and Requirements,0.307692,# Grant Application Entities and Requirements\...,7.5
8,25,Canadian Businesses and COVID-19 Support Commu...,0.307692,# Canadian Businesses and COVID-19 Support Com...,7.5
9,17,Canada Small Business Financing Program and Go...,0.307692,# Canada Small Business Financing Program and ...,7.5


## Local Search

In [8]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore
from dotenv import load_dotenv

load_dotenv()

INPUT_DIR = "C:/Users/rishu/OneDrive/Desktop/Rishub/AiHCCC/agents-grant/grant_agent_2/graphrag/ragtest/output/20240704-184122/artifacts"
LANCEDB_URI = "C:/Users/rishu/OneDrive/Desktop/Rishub/AiHCCC/agents-grant/grant_agent_2/graphrag/lancedb"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

In [9]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 318


,level,title,type,description,source_id,degree,human_readable_id,id,size,graph_embedding,community,entity_type,top_level_node_id,x,y
0,0,"""CIHR PROJECT GRANT""","""GRANT_CATEGORY""","""CIHR Project Grant is a grant category that s...",93dd9657e2ca2191832253675ec7701d,2,0,b45241d70f0e43fca764df95b2b81f77,2,None,None,None,b45241d70f0e43fca764df95b2b81f77,0,0
1,0,"""CIHR FOUNDATION GRANT""","""GRANT_CATEGORY""",The CIHR Foundation Grant is a grant category ...,"93dd9657e2ca2191832253675ec7701d,edd71dfd82264...",3,1,4119fd06010c494caa07f439b333f4c5,3,None,None,None,4119fd06010c494caa07f439b333f4c5,0,0
2,0,"""CANADA""","""GEO""","""CANADA"" is a country known for providing vari...","05a4cfd3c4edb4947d01f9d9e7149d93,358f7c199dbcd...",7,2,d3835bf3dda84ead99deadbeac5d0d7d,7,None,3,None,d3835bf3dda84ead99deadbeac5d0d7d,0,0
3,0,"""HEALTH-RELATED RESEARCH PROJECTS""","""DOMAIN""","""Health-related Research Projects are the focu...",93dd9657e2ca2191832253675ec7701d,1,3,077d2820ae1845bcbb1803379a3d1eae,1,None,None,None,077d2820ae1845bcbb1803379a3d1eae,0,0
4,0,"""LONG-TERM HEALTH RESEARCH PROJECTS""","""DOMAIN""","""Long-term Health Research Projects are the fo...",93dd9657e2ca2191832253675ec7701d,1,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,1,None,None,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,0


In [10]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 159


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,"""CIHR PROJECT GRANT""","""HEALTH-RELATED RESEARCH PROJECTS""",1.0,"""CIHR Project Grant supports projects in the d...",[93dd9657e2ca2191832253675ec7701d],cbf232211e7d4eb6abdbe182f71c2cf0,0,2,1,3
1,"""CIHR PROJECT GRANT""","""CANADIAN INSTITUTES OF HEALTH RESEARCH (CIHR)""",1.0,"""CIHR Project Grant is sponsored by the Canadi...",[93dd9657e2ca2191832253675ec7701d],bb0cff774a4440b289cc6f3b929fe13c,1,2,2,4
2,"""CIHR FOUNDATION GRANT""","""LONG-TERM HEALTH RESEARCH PROJECTS""",1.0,"""CIHR Foundation Grant supports projects in th...",[93dd9657e2ca2191832253675ec7701d],ce55841ebfdd47008bab8c258f10372e,2,3,1,4
3,"""CIHR FOUNDATION GRANT""","""CANADIAN INSTITUTES OF HEALTH RESEARCH (CIHR)""",1.0,"""CIHR Foundation Grant is sponsored by the Can...",[93dd9657e2ca2191832253675ec7701d],6090e736374d45fd84f0e4610a314f8f,3,3,2,5
4,"""CIHR FOUNDATION GRANT""","""CIHR""",1.0,"""CIHR Foundation Grant is sponsored by CIHR an...",[edd71dfd82264259bc887de72dd5a28c],0e8d921ccd8d4a8594b65b7fd19f7120,4,3,1,4


In [11]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 32


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,10,# SSHRC Doctoral Fellowships and Research Prop...,1,7.5,SSHRC Doctoral Fellowships and Research Proposals,The impact severity rating is high due to the ...,The community revolves around SSHRC Doctoral F...,[{'explanation': 'SSHRC Doctoral Fellowships p...,"{\n ""title"": ""SSHRC Doctoral Fellowships an...",a97f9429-b24b-4d3f-81b7-bce2bfa5a865
1,11,# Ontario Graduate Scholarship and Academic Tr...,1,7.5,Ontario Graduate Scholarship and Academic Tran...,The impact severity rating is high due to the ...,The community is centered around the Ontario G...,[{'explanation': 'The Ontario Graduate Scholar...,"{\n ""title"": ""Ontario Graduate Scholarship ...",0684409b-fac7-4ac6-803a-407a8f6ef84b
2,12,# Canadian Liver Foundation and Health Grants\...,1,7.5,Canadian Liver Foundation and Health Grants,The impact severity rating is high due to the ...,The community is centered around the Canadian ...,[{'explanation': 'The Canadian Liver Foundatio...,"{\n ""title"": ""Canadian Liver Foundation and...",414e537f-1b23-4f2a-b131-6d9ec3884c4b
3,13,# Indigenous Languages and Cultures Program Co...,1,7.5,Indigenous Languages and Cultures Program Comm...,The impact severity rating is high due to the ...,The Indigenous Languages and Cultures Program ...,[{'explanation': 'The Indigenous Languages and...,"{\n ""title"": ""Indigenous Languages and Cult...",fb3c0ebc-c983-4eb9-b1b4-d8b41ae7826c
4,14,# Canadian Heritage and Cultural Funding\n\nTh...,1,8.5,Canadian Heritage and Cultural Funding,The impact severity rating is high due to the ...,The community revolves around Canadian Heritag...,[{'explanation': 'Canadian Heritage offers a w...,"{\n ""title"": ""Canadian Heritage and Cultura...",a15a6886-81c0-49e0-8c1e-25325eb720f3


In [12]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 41


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,93dd9657e2ca2191832253675ec7701d,Here is a table with the top 10 healthcare gra...,300,[1fe6411b44bd3f8a68162404a0304c73],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[cbf232211e7d4eb6abdbe182f71c2cf0, bb0cff774a4..."
1,edd71dfd82264259bc887de72dd5a28c,Supports a diverse portfolio of health-relate...,300,[1fe6411b44bd3f8a68162404a0304c73],"[4119fd06010c494caa07f439b333f4c5, e7ffaee9d31...","[0e8d921ccd8d4a8594b65b7fd19f7120, c8b24086178..."
2,1f38035eafa84974509be10b22e3be69,Healthcare | Focuses on ...,300,[1fe6411b44bd3f8a68162404a0304c73],"[f7e11b0e297a44a896dc67928368f600, 27f9fbe6ad8...","[a5e0d1644eb547ba9a5c3211aac4631a, 5a28b94bc63..."
3,ebbba0493660fa709492f99aac27f776,Canada Foundation for Innovation | Support...,300,[1fe6411b44bd3f8a68162404a0304c73],"[96aad7cb4b7d40e9b7e13b94a67af206, 9646481f66c...","[f97011b2a99d44648e18d517e1eae15c, 35489ca6a63..."
4,74ba050c6bacf63cc77b1946a9fe3228,& Stroke Foundation ...,300,[1fe6411b44bd3f8a68162404a0304c73],"[254770028d7a4fa9877da4ba0ad5ad21, 4a67211867e...","[5d3344f45e654d2c808481672f2f08dd, 6fb57f83bae..."


In [13]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

In [14]:
context_builder_local = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder_local,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [15]:
result = await search_engine.asearch("Can you tell me the best grants for women in healthcare sector?")
print(result.response)
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")
result.context_data["reports"]

When it comes to grants specifically tailored for women in the healthcare sector, there are several notable opportunities available. One prominent grant is the "Women Entrepreneurship Fund," which is designed to support the growth and expansion of businesses owned or led by women. This fund provides grants of up to $100,000 to eligible women entrepreneurs, aiming to facilitate their business development [Data: Entities (52); Relationships (155); Sources (9)]. Another significant grant is the "Heart & Stroke Foundation Grant," which varies in amount and supports cardiovascular and cerebrovascular research. This grant requires a research proposal, budget, CV, and reference letters, emphasizing its focus on advancing research in heart disease and stroke [Data: Entities (16); Relationships (54)].

Moreover, the "Canadian Cancer Society Innovation Grant" offers funding of up to $100,000 for innovative cancer research projects. This grant is specifically geared towards Canadian cancer resear

,id,title,content
0,22,Educational Grants and CDHA,# Educational Grants and CDHA\n\nThe community...
1,23,CDHA Community and Opportunities,# CDHA Community and Opportunities\n\nThe CDHA...


In [16]:
result = await search_engine.asearch("Can you write a grant proposal for a small business in the health care sector? like the CIHR foundation grant.")
print(result.response)
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")
result.context_data["reports"]

# Grant Proposal: Small Business in the Healthcare Sector

## Introduction
Our small business, dedicated to advancing healthcare solutions, is seeking funding through the CIHR Foundation Grant to support our innovative projects in the health sector. With a focus on improving health outcomes and driving impactful research, our business aims to make significant contributions to the healthcare industry.

## Project Description
The proposed project involves the development of a novel healthcare technology that enhances patient care and treatment outcomes. By leveraging cutting-edge research and technology, our business aims to address critical challenges in the healthcare sector, ultimately improving the quality of care and patient outcomes.

## Objectives
1. Develop and implement a prototype of the healthcare technology.
2. Conduct rigorous testing and validation to ensure the technology's effectiveness.
3. Collaborate with healthcare professionals to integrate the technology into clinica

,id,title,content
0,17,Canada Small Business Financing Program and Go...,# Canada Small Business Financing Program and ...
1,12,Canadian Liver Foundation and Health Grants,# Canadian Liver Foundation and Health Grants\...


In [17]:
result.context_data["entities"].head()

,id,entity,description,number of relationships,in_context
0,1,"""CIHR FOUNDATION GRANT""",The CIHR Foundation Grant is a grant category ...,3,True
1,0,"""CIHR PROJECT GRANT""","""CIHR Project Grant is a grant category that s...",2,True
2,41,"""CANADA SMALL BUSINESS FINANCING PROGRAM""","The ""CANADA SMALL BUSINESS FINANCING PROGRAM"" ...",2,True
3,45,"""SMALL BUSINESSES""","""Small businesses with gross annual revenues o...",1,True
4,73,"""SMALL BUSINESS GRANTS""",,1,True


In [18]:
result.context_data["relationships"].head()

,id,source,target,description,weight,rank,links,in_context
0,61,"""CANADA SMALL BUSINESS FINANCING PROGRAM""","""GOVERNMENT OF CANADA""","The Canada Small Business Financing Program, s...",2.0,24,1,True
1,25,"""CANADIAN LIVER FOUNDATION""","""RESEARCH PROPOSAL""","""A research proposal is a required document fo...",1.0,9,1,True
2,24,"""CANADIAN LIVER FOUNDATION""","""HEALTH""","""The Canadian Liver Foundation supports resear...",1.0,7,1,True
3,3,"""CIHR FOUNDATION GRANT""","""CANADIAN INSTITUTES OF HEALTH RESEARCH (CIHR)""","""CIHR Foundation Grant is sponsored by the Can...",1.0,5,2,True
4,1,"""CIHR PROJECT GRANT""","""CANADIAN INSTITUTES OF HEALTH RESEARCH (CIHR)""","""CIHR Project Grant is sponsored by the Canadi...",1.0,4,2,True


In [19]:
result.context_data["reports"].head()

,id,title,content
0,17,Canada Small Business Financing Program and Go...,# Canada Small Business Financing Program and ...
1,12,Canadian Liver Foundation and Health Grants,# Canadian Liver Foundation and Health Grants\...


In [20]:
result.context_data["sources"].head()

,id,text
0,0,Here is a table with the top 10 healthcare gra...
1,1,Supports a diverse portfolio of health-relate...
2,8,| Government of Canada | Cove...
3,33,Here is a table with the top 10 entrepreneursh...
4,34,| Government of Canada | Cove...


In [13]:
'''Search for grants based on the following requirements: grant_amount=250000 USD 
eligibility_criteria=['Foreign National'] 
grant_category=Business 
project_name=Alpha Dating App 
project_description=Connect straight people with like-minded individuals for dating and marriage assistance, including wedding planning services. 
project_usecase=Primary use case is to connect straight individuals for dating and offer comprehensive wedding planning services. Additionally, an Advisory Council will help couples with pre-marriage compatibility checks and ongoing marital support. 
project_outcomes=Increased marriage rates and decreased divorce rates.
project_execution_plan=1-year development phase, 6 months of beta testing, and then public release of the product.

Write a sample Grant Drafts for all the requirements mentioned above for me to submit a grant.'''

"Search for grants based on the following requirements: grant_amount=250000 USD \neligibility_criteria=['Foreign National'] \ngrant_category=Business \nproject_name=Alpha Dating App \nproject_description=Connect straight people with like-minded individuals for dating and marriage assistance, including wedding planning services. \nproject_usecase=Primary use case is to connect straight individuals for dating and offer comprehensive wedding planning services. Additionally, an Advisory Council will help couples with pre-marriage compatibility checks and ongoing marital support. \nproject_outcomes=Increased marriage rates and decreased divorce rates.\nproject_execution_plan=1-year development phase, 6 months of beta testing, and then public release of the product.\n\nWrite a sample Grant Drafts for all the requirements mentioned above for me to submit a grant."